As I don't yet have found a dataset with more features, I will start to implement the script with the solvent dataset.

In [1]:
import pandas as pd
import numpy as np
#import rdkit

%matplotlib inline

### Loading the data

In [2]:
dataset = pd.read_csv('../datasets/FreeSolv/SAMPL.csv')

In [3]:
type(dataset)

pandas.core.frame.DataFrame

In [4]:
dataset.head()

,iupac,smiles,expt,calc
0,"4-methoxy-N,N-dimethyl-benzamide",CN(C)C(=O)c1ccc(cc1)OC,-11.01,-9.625
1,methanesulfonyl chloride,CS(=O)(=O)Cl,-4.87,-6.219
2,3-methylbut-1-ene,CC(C)C=C,1.83,2.452
3,2-ethylpyrazine,CCc1cnccn1,-5.45,-5.809
4,heptan-1-ol,CCCCCCCO,-4.21,-2.917


### Data Exploration

In [5]:
df = dataset.drop(columns = ['iupac'])

In [6]:
df.describe()

,expt,calc
count,642.000000,642.000000
mean,-3.803006,-3.486223
std,3.847820,4.186301
min,-25.470000,-21.762000
25%,-5.727500,-5.421500
50%,-3.530000,-3.053500
75%,-1.215000,-0.480750
max,3.430000,3.335000


I'll take a small subset for now.

In [7]:
df = df.sample(n = 20)
df

,smiles,expt,calc
14,c1ccc(cc1)S,-2.55,-1.501
45,CCCC(=O)OC,-2.83,-3.552
191,COP(=S)(OC)SCn1c(=O)c2ccccc2nn1,-10.03,-14.106
366,Cc1ccccc1N,-5.53,-5.325
573,CCC[C@@H](C)O,-4.39,-2.945
598,c1cc2ccc3cccc4c3c2c(c1)cc4,-4.52,-6.790
534,CCCl,-0.63,0.775
157,CCCCCC(=O)O,-6.21,-7.878
320,CCCc1ccc(c(c1)OC)O,-5.26,-4.127
262,COC(OC)OC,-4.42,-4.625


## Transform SMILES to Fingerprints

Still to answer: Are these SMILES canonical or not?
I would love to transform the SMILES to [Extended Connectivity Fingerprints (ECFPs)](https://docs.chemaxon.com/display/docs/Extended+Connectivity+Fingerprint+ECFP). However, I can't find a way to do that so far. The methods I find are either terminal (I assume, but don't know for sure) or in Java. (ECFP = Morgan fingerprint)

But I found a way for [Morgan fingerprints](http://www.rdkit.org/docs/GettingStartedInPython.html). [Here's](http://bigchem.eu/sites/default/files/Martin_Vogt_algorithms_in_cheminformatics_150519.pdf) how Morgan Fingerprints ("circular Fingerprints") work.
Nevermind.

My next try: creating Rdkit fingerprints. [This blog](https://iwatobipen.wordpress.com/2019/02/08/convert-fingerprint-to-numpy-array-and-conver-numpy-array-to-fingerprint-rdkit-memorandum/) is superb.

there is [MolFromSmiles](https://www.rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html) -> Construct a molecule from a SMILES string

[FingerprintsfromSmiles](https://www.rdkit.org/docs/source/rdkit.Chem.Fingerprints.FingerprintMols.html)

In [8]:
#from rdkit.Chem.Fingerprints import FingerprintMol
#from rdkit import Chem
#from rdkit.Chem import Fingerprints

import rdkit.Chem.Fingerprints.FingerprintMols as fp_from_smiles

In [25]:
df['id'] = range(len(df))    # the function requires an id column
#df['id'] = pd.to_numeric(range(len(df)))   no.

In [36]:
my_fingerprints = fp_from_smiles.FingerprintsFromSmiles(dataSource=df, idCol='id', smiCol='smiles')
#my_fingerprints = fp_from_smiles.FingerprintsFromSmiles(dataSource=df, idCol=df.index, smiCol='smiles')

TypeError: string indices must be integers

arguments:    dataSource, idCol, smiCol, fingerprinter=<Boost.Python.function object>, reportFreq=10, maxMols=-1, \**fpArgs

fpArgs are passed as keyword arguments to the fingerprinter\
Returns a list of 2-tuples: (ID,fp)


## Generate a umap of the fingerprints

https://pypi.org/project/umap-learn/

https://umap-learn.readthedocs.io/en/latest/basic_usage.html

I need more physiochemical properties of course..

In [32]:
df.index

Int64Index([ 14,  45, 191, 366, 573, 598, 534, 157, 320, 262, 629, 392,  99,
            627, 569, 468, 347, 464, 375, 297],
           dtype='int64')